# Import Required Libraries
Import the necessary libraries, including pandas and json.

In [6]:
# Import the necessary libraries, including pandas and json
import pandas as pd
import json
import os

# Change year to the year you want
year = 2022

In [7]:
# Define the path to the folder containing files
folder_path = f'Project/{year}'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith(str(year)) and not filename.endswith('.json'):
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, filename + '.json')
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed {old_file_path} to {new_file_path}")

print("Renaming completed.")

Renamed Project/2022\202200000 to Project/2022\202200000.json
Renamed Project/2022\202200001 to Project/2022\202200001.json
Renamed Project/2022\202200002 to Project/2022\202200002.json
Renamed Project/2022\202200003 to Project/2022\202200003.json
Renamed Project/2022\202200004 to Project/2022\202200004.json
Renamed Project/2022\202200005 to Project/2022\202200005.json
Renamed Project/2022\202200006 to Project/2022\202200006.json
Renamed Project/2022\202200007 to Project/2022\202200007.json
Renamed Project/2022\202200008 to Project/2022\202200008.json
Renamed Project/2022\202200009 to Project/2022\202200009.json
Renamed Project/2022\202200010 to Project/2022\202200010.json
Renamed Project/2022\202200011 to Project/2022\202200011.json
Renamed Project/2022\202200012 to Project/2022\202200012.json
Renamed Project/2022\202200013 to Project/2022\202200013.json
Renamed Project/2022\202200014 to Project/2022\202200014.json
Renamed Project/2022\202200015 to Project/2022\202200015.json
Renamed 

# Load JSON File
Load the JSON file from the specified path.

In [8]:
import os
import json
import pandas as pd

# Path to the folder containing JSON files
folder_path = f'Project/{year}'

# Initialize a list to store the rows of the DataFrame
data_rows = []

# Columns to extract
columns_to_keep = [
    'coredata.dc:title',
    'coredata.prism:publicationName',
    'item.bibrecord.head.citation-info.citation-language.@language',
    'coredata.citedby-count'
]

# Helper function to safely extract a value from a dictionary
def safe_get(d, keys, default=None):
    try:
        for key in keys:
            d = d.get(key, {})
        return d if d else default
    except AttributeError:
        return default

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith(str(year)) and file_name.endswith(".json"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)

                # Extract countries from author-group
                author_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], [])
                countries = [
                    safe_get(author, ['affiliation', 'country'], 'Unknown')
                    for author in author_groups
                ]
                countries_string = ','.join(countries)

                # Extract keywords
                auth_keywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords', 'author-keyword'], [])
                keywords = [keyword.get('$', '') for keyword in auth_keywords if isinstance(keyword, dict)]
                keywords_string = ','.join(keywords)

                # Normalize JSON data and filter columns
                row_data = pd.json_normalize(data.get('abstracts-retrieval-response', {}))
                row = {col: row_data[col].iloc[0] if col in row_data else pd.NA for col in columns_to_keep}

                # Add processed fields
                row['item.bibrecord.head.author-group.affiliation.country'] = countries_string
                row['authkeywords.author-keyword'] = keywords_string

                # Append the row to data_rows
                data_rows.append(row)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)

# Save the DataFrame to a CSV file
output_path = f'output_{year}.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

# Display the DataFrame for verification
print(df)


                                      coredata.dc:title  \
0     Reduction of algal wall-growth biofilm for the...   
1     Dietary anthocyanins inhibit insulin fibril fo...   
2     Metformin delivery via iontophoresis based on ...   
3     Experimental investigations of CO2 absorption ...   
4     High-throughput screening of highly active and...   
...                                                 ...   
4239  Executing trades in style: retail investors vs...   
4240  Placental Findings Contributing to Perinatal D...   
4241  Determination of the ash content of coal sampl...   
4242  Multi-criteria rating conversion without relat...   
4243  Linking knowledge enabling factors to organiza...   

                         coredata.prism:publicationName  \
0                       Biochemical Engineering Journal   
1     International Journal of Biological Macromolec...   
2     International Journal of Biological Macromolec...   
3                          Chemical Engineering Science

In [55]:
import json
import pandas as pd

# Define the path to the JSON file
json_file_path = 'Project/2023/202302889.json'

# Open and load the JSON file
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract countries from author-group
countries = [author['affiliation']['country'] for author in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']]

# Convert the list of countries to a single string
countries_string = ','.join(countries)

# Extract $ values from authkeywords.author-keyword
keywords = [keyword['$'] for keyword in data['abstracts-retrieval-response']['authkeywords']['author-keyword']]

# Convert the list of keywords to a single string
keywords_string = ','.join(keywords)

# Normalize JSON data to a DataFrame
df = pd.json_normalize(data['abstracts-retrieval-response'])

# Define columns to keep
columns_to_keep = [
    'coredata.dc:title',
    'coredata.prism:publicationName',
    'item.bibrecord.head.citation-info.citation-language.@language',
    'coredata.citedby-count'
]

# Add the countries_string and keywords_string to the DataFrame
df['item.bibrecord.head.author-group.affiliation.country'] = countries_string
df['authkeywords.author-keyword'] = keywords_string

# Keep only the specified columns
df = df[columns_to_keep + ['item.bibrecord.head.author-group.affiliation.country', 'authkeywords.author-keyword']]


df.to_csv('output.csv', index=False)

# Display the DataFrame to verify the contents
df

,coredata.dc:title,coredata.prism:publicationName,item.bibrecord.head.citation-info.citation-language.@language,coredata.citedby-count,item.bibrecord.head.author-group.affiliation.country,authkeywords.author-keyword
0,Effects of remittances on household poverty an...,Journal of the Asia Pacific Economy,English,3,"Cambodia,Thailand","Cambodia,consumption,expenditure,Impact,inequa..."
